### Prebuilt react agent with search tool and memory

In [1]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import BaseMessage
from langgraph.prebuilt import create_react_agent

/Users/rickchakra/Projects/aiug_workshop/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Create the agent with memory
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(model='gpt-4o')
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [3]:
# Use the agent with specific thread id (memory)
config = {"configurable": {"thread_id": "user2"}}

In [4]:
# pass in config and call agent_executor.stream to interact with the agent
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in agent_executor.stream({"messages": [("user", user_input)]}, config):
        print(event)
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xZBlltZ83Br36xREAorGybKI', 'function': {'arguments': '{"query":"Los Angeles weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_400f27fa1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fe1ac473-8b59-44c0-8d05-14051f5bf11a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather today'}, 'id': 'call_xZBlltZ83Br36xREAorGybKI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87, 'output_tokens': 21, 'total_tokens': 108})]}}
Assistant: 
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 

In [5]:
# check what's stored in memory
agent_executor.get_state(config).values['messages']

[HumanMessage(content='Hi', id='d848c185-b0dd-4a3a-bc24-17a739c63ac7'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 80, 'total_tokens': 90}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'stop', 'logprobs': None}, id='run-5cde0d8f-fcee-4239-b7a3-eead114b2b56-0', usage_metadata={'input_tokens': 80, 'output_tokens': 10, 'total_tokens': 90}),
 HumanMessage(content='my name is Rick', id='ffaba971-a8ca-475f-82dc-5a03d56bddc7'),
 AIMessage(content='Nice to meet you, Rick! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 101, 'total_tokens': 116}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5e997b69d8', 'finish_reason': 'stop', 'logprobs': None}, id='run-30f85200-5a85-4136-941b-2e064ae5643d-0', usage_metadata={'input_tokens': 101, 'output_tokens': 15, 'total_tokens': 116}),
 HumanMessage(